In [28]:
# https://medium.com/@erikhallstrm/hello-world-rnn-83cd7105b767
DEBUG = True

import re, random, math, csv, io, string, itertools, sys, functools,math
import numpy as np
# import pandas as pd
import tensorflow as tf

def test_data_1(n_unrollings=100, echo_step=2, batch_size=5):
    x = np.array(np.random.choice(2, n_unrollings, p=[0.5, 0.5]))
    y = np.roll(x, echo_step)
    y[0:echo_step] = 0

    x = x.reshape((batch_size, -1))  # The first index changing slowest, subseries as rows
    y = y.reshape((batch_size, -1))

    return (x, y)

class RNN:
    hp = dict(
        batch_sz=5,
        backprop_len=15,
        n_classes=2,
        state_sz=4,
        ckpt_path="./checkpoints/"
    )
    def __init__(self, **hyper_parameters):
        if hyper_parameters is not None:
            for k, v in hyper_parameters.items():
                self.hp[k] = v
    
    def _init_graph(self):
        tf.reset_default_graph()
        self.inputs_batch = tf.placeholder(tf.float32, [self.hp['batch_sz'], self.hp['backprop_len']], "inputs_batch")
        self.labels_batch = tf.placeholder(tf.int32,   [self.hp['batch_sz'], self.hp['backprop_len']],   "labels_batch")
        self.init_state = tf.placeholder(tf.float32,   [self.hp['batch_sz'], self.hp['state_sz']],     "init_state")
        
        self.w_tarns = tf.Variable(
            np.random.rand(
                self.hp['state_sz']+1, #prev_state + 1 for inputs
                self.hp['state_sz']
            ),
            dtype=tf.float32
        )
        self.b_trans = tf.Variable(np.zeros([1,self.hp['state_sz']]), dtype=tf.float32)
        self.w_out = tf.Variable(np.random.rand(self.hp["state_sz"], self.hp["n_classes"]), dtype=tf.float32)
        self.b_out = tf.Variable(np.zeros([1,self.hp['n_classes']]), dtype=tf.float32)
        
        self.inputs_seq = tf.unstack(self.inputs_batch, axis=1)
        self.labels_seq = tf.unstack(self.labels_batch, axis=1)

    def _forward(self):
        curr_state = self.init_state
        state_seq = []
        for curr_input in self.inputs_seq:
            curr_input = tf.reshape(curr_input, [self.hp['batch_sz'], 1])
            comm = tf.concat([curr_input, curr_state], axis=1)
            next_state = tf.tanh(tf.matmul(comm, self.w_tarns) + self.b_trans)
            state_seq.append(next_state)
            curr_state = next_state
            
        self.last_state = curr_state
        self.logits_seq = [tf.matmul(state, self.w_out)+self.b_out for state in state_seq]
        self.preds_seq = [tf.nn.softmax(logit) for logit in self.logits_seq]
        
    def _train_graph(self):
        self._init_graph()
        self._forward()
        
        # backward
        losses = [
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits = logits,
                labels = labels
            ) 
            for logits, labels in zip(self.logits_seq, self.labels_seq)
        ]
        self.total_loss = tf.reduce_mean(losses)
        self.train_step = tf.train.AdagradOptimizer(0.3).minimize(self.total_loss)
        tf.summary.histogram(self.total_loss.op.name, self.total_loss)
        tf.summary.histogram(self.last_state.op.name, self.last_state)
        self.merged = tf.summary.merge_all()
    
    def _infer_graph(self):
        self._init_graph()
        self._forward()
        
        
    def predict(self, x):
        nobs = np.shape(x)[0]
        self._infer_graph()
        prediction=[]
        if nobs<self.hp["backprop_len"]:
            x = np.pad(x, (0, self.hp["backprop_len"] - nobs),mode="constant")
        with tf.Session() as sess:
            sess.run(tf.initialize_all_variables())
            ckpt = tf.train.get_checkpoint_state("checkpoints")
            n_batches = 1 + int(nobs>self.hp['backprop_len'])*(nobs - self.hp['backprop_len'])
            start_idx = 0
            end_idx = self.hp["backprop_len"]
            curr_state = np.zeros([1, self.hp['state_sz']])
            is_first = True
            for batch_num in range(n_batches):
                x_batch = [x[start_idx:]]
                pred = sess.run(
                    [self.preds_seq],
                    feed_dict={
                        self.inputs_batch:x_batch,
                        self.init_state:curr_state
                    }
                )
                if is_first:
                    prediction.extend(pred)
                    is_first=False
                else:
                    prediction.append(pred[-1])
        return prediction[0][:nobs]
    
    def train(self, x, y,n_epochs=10):
        nobs = len(x)
        divider = self.hp['batch_sz']*self.hp["backprop_len"]*n_epochs
        pad_len = divider - (nobs%divider)
        
        if nobs<pad_len:
            raise Exception("Wrong n_epochs({n_epochs}): choose smaller value.")
        x_batch=np.reshape(
            np.pad(x, (0, pad_len), mode='constant'),
            (n_epochs,self.hp['batch_sz'], -1, self.hp["backprop_len"])
        )
        y_batch=np.reshape(
            np.pad(y, (0, pad_len), mode='constant'),
            (n_epochs,self.hp['batch_sz'], -1, self.hp["backprop_len"])
        )
        n_batches=x_batch.shape[2]
        print(f"Started trainning: n_epochs={n_epochs} n_batches={n_batches}")
        self._train_graph()
        saver = tf.train.Saver()
        with tf.Session() as sess:
            
            train_writer = tf.summary.FileWriter( './mdl_tr', sess.graph)
#             sess.run(tf.initialize_all_variables())
            sess.run(tf.global_variables_initializer())
            losses=[]
            for epoch_num in range(n_epochs):
                curr_state = np.zeros([self.hp['batch_sz'], self.hp['state_sz']])
                for batch_idx in range(n_batches):
                    start = batch_idx*self.hp['batch_sz']
                    finish = start + self.hp['batch_sz']
                    summary, train_step, loss, curr_state, preds = sess.run(
#                     train_step, loss, curr_state, preds = sess.run(
                        [self.merged, self.train_step, self.total_loss, self.last_state, self.preds_seq],
#                         [self.train_step, self.total_loss, self.last_state, self.preds_seq],
                        feed_dict={
                            self.inputs_batch:x_batch[epoch_num,:,batch_idx,:],
                            self.labels_batch:y_batch[epoch_num,:,batch_idx,:],
                            self.init_state:curr_state
                        }
                    )
                    if batch_idx%100==0:
#                         train_writer.add_summary(summary, global_step=epoch_num)
                        saver.save(sess, self.hp['ckpt_path'] + "nn.mdl", global_step=epoch_num)
                        ypred = [1 if x[0]<0.5 else 0 for pred in preds for x in pred]
                        ytrue = [x for pred in y_batch[epoch_num,:,batch_idx,:] for x in pred]
                        print(f"epoch={epoch_num} batch_idx={batch_idx}")
                        print(f"ytrue={y_batch[epoch_num,0, batch_idx, :]}")
                        print(f"ypred={[x[1] for x in np.array(preds)[:,0,:]]}")
                        print(f"loss={loss}")

            
echo_step = 1
x = np.array(np.random.choice(2, 500000, p=[0.5, 0.5]))
y = np.roll(x, echo_step)
y[0:echo_step] = 0

rnn = RNN()
rnn.train(x, y, n_epochs=50)

Started trainning: n_epochs=50 n_batches=134
epoch=0 batch_idx=0
ytrue=[0 1 1 1 0 0 1 0 1 1 1 1 0 1 1]
ypred=[0.45999932, 0.35109955, 0.32599562, 0.32812956, 0.32860422, 0.3241758, 0.32795367, 0.32412425, 0.32368734, 0.32366654, 0.32366535, 0.32791322, 0.32411993, 0.32368705, 0.32366654]
loss=0.8442586064338684
epoch=0 batch_idx=100
ytrue=[1 0 0 1 1 0 0 1 0 0 1 0 1 0 0]
ypred=[0.5470528, 0.54718423, 0.54664987, 0.54648805, 0.54705292, 0.54718423, 0.54664981, 0.54705554, 0.54718512, 0.54664999, 0.54705566, 0.54663616, 0.54705518, 0.547185, 0.54719418]
loss=0.7088890671730042
epoch=1 batch_idx=0
ytrue=[0 0 1 0 0 0 1 0 1 1 1 1 1 1 1]
ypred=[0.51728433, 0.5503785, 0.51419729, 0.51408243, 0.51190412, 0.51120895, 0.51147914, 0.51114595, 0.51099622, 0.51099068, 0.51099038, 0.51099038, 0.51099038, 0.51099038, 0.51099032]
loss=0.695429801940918
epoch=1 batch_idx=100
ytrue=[1 1 1 1 0 1 1 1 1 1 0 0 1 0 0]
ypred=[0.50095475, 0.50095487, 0.50095493, 0.50102472, 0.50105929, 0.50095826, 0.5009551, 0.

epoch=15 batch_idx=100
ytrue=[1 1 0 1 1 1 1 0 1 1 1 1 1 1 1]
ypred=[0.99889249, 0.99930012, 0.00089196383, 0.99881941, 0.99890661, 0.99889344, 0.99929965, 0.00089133641, 0.99881881, 0.99890685, 0.99889332, 0.99890494, 0.99889874, 0.99890232, 0.99929488]
loss=0.0009645065874792635
epoch=16 batch_idx=0
ytrue=[0 0 0 0 0 0 0 1 0 1 1 1 0 0 1]
ypred=[0.26245639, 0.00043027746, 0.0047450964, 0.00062436261, 0.0016065127, 0.00079591875, 0.0011027645, 0.99941719, 0.0010954875, 0.99900597, 0.99882454, 0.99936408, 0.001034653, 0.00082859892, 0.99923325]
loss=0.03531138598918915
epoch=16 batch_idx=100
ytrue=[0 0 1 0 1 1 0 1 0 1 1 0 0 1 1]
ypred=[0.00091031846, 0.00079186197, 0.99931216, 0.00078527821, 0.99890816, 0.99938035, 0.00082942174, 0.99934226, 0.00082139287, 0.99894065, 0.99936205, 0.00091976259, 0.00080395793, 0.99892694, 0.9993698]
loss=0.0008508177706971765
epoch=17 batch_idx=0
ytrue=[0 1 1 1 1 1 0 1 1 0 1 1 1 0 0]
ypred=[0.330015, 0.99958855, 0.99386483, 0.99938059, 0.99839562, 0.999551

epoch=30 batch_idx=100
ytrue=[0 0 0 0 1 0 0 0 1 0 0 0 1 0 0]
ypred=[0.00045418297, 0.00043443911, 0.00043834274, 0.00032206104, 0.99971968, 0.0004491696, 0.00043779844, 0.00032327126, 0.99972063, 0.00045005445, 0.00043719602, 0.00032374499, 0.99972099, 0.00045042078, 0.00043694771]
loss=0.00037074272404424846
epoch=31 batch_idx=0
ytrue=[0 1 1 1 1 0 0 0 0 0 0 1 0 0 1]
ypred=[0.55703044, 0.99982494, 0.99731809, 0.99974018, 0.99937207, 0.00031095155, 0.00062245596, 0.00036343964, 0.00049376872, 0.00040033527, 0.00034745724, 0.99974269, 0.00046465354, 0.00030284142, 0.99956125]
loss=0.04934898018836975
epoch=31 batch_idx=100
ytrue=[0 1 1 0 0 0 0 0 0 1 1 0 0 0 1]
ypred=[0.00028592735, 0.99954969, 0.99975485, 0.00044259868, 0.00040916982, 0.00042227108, 0.00041329899, 0.00041891905, 0.00029457294, 0.99956316, 0.99974686, 0.00043491792, 0.00041399713, 0.00029908092, 0.99972147]
loss=0.00035299320006743073
epoch=32 batch_idx=0
ytrue=[0 0 1 1 1 0 0 0 0 1 0 1 0 1 0]
ypred=[0.36965698, 7.7893019e

epoch=44 batch_idx=100
ytrue=[1 1 1 1 0 0 0 1 1 0 1 0 0 0 0]
ypred=[0.99974912, 0.99974817, 0.999749, 0.99982303, 0.00030213894, 0.0002919824, 0.00019836778, 0.99974245, 0.9998228, 0.00020589038, 0.99981767, 0.00030033305, 0.00029315922, 0.00029204835, 0.00029230744]
loss=0.0002385669940849766
epoch=45 batch_idx=0
ytrue=[0 0 1 0 1 0 1 1 1 1 1 1 1 1 1]
ypred=[0.5081529, 4.8759943e-05, 0.99872476, 5.9808193e-05, 0.99907804, 6.6501387e-05, 0.99947518, 0.99980956, 0.99968708, 0.99977642, 0.99973005, 0.99976069, 0.99974304, 0.99975425, 0.99982053]
loss=0.06552381813526154
epoch=45 batch_idx=100
ytrue=[1 1 1 0 1 0 1 0 1 1 1 1 1 1 0]
ypred=[0.99971849, 0.99975151, 0.99982089, 0.00017467602, 0.99979931, 0.00015981452, 0.99977893, 0.00014592728, 0.99968696, 0.99976557, 0.99972516, 0.99975175, 0.99973613, 0.99983275, 0.00029339435]
loss=0.00022581909433938563
epoch=46 batch_idx=0
ytrue=[0 0 1 0 1 1 1 0 1 1 1 0 0 1 0]
ypred=[0.46304005, 4.6950503e-05, 0.99871206, 5.640529e-05, 0.99934822, 0.99981

In [12]:
from __future__ import print_function, division
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

num_epochs = 100
total_series_length = 50000
truncated_backprop_length = 15
state_size = 4
num_classes = 2
echo_step = 3
batch_size = 5
num_batches = total_series_length//batch_size//truncated_backprop_length

def plot(loss_list, predictions_series, batchX, batchY):
    plt.subplot(2, 3, 1)
    plt.cla()
    plt.plot(loss_list)

    for batch_series_idx in range(5):
        one_hot_output_series = np.array(predictions_series)[:, batch_series_idx, :]
        single_output_series = np.array([(1 if out[0] < 0.5 else 0) for out in one_hot_output_series])

        plt.subplot(2, 3, batch_series_idx + 2)
        plt.cla()
        plt.axis([0, truncated_backprop_length, 0, 2])
        left_offset = range(truncated_backprop_length)
        plt.bar(left_offset, batchX[batch_series_idx, :], width=1, color="blue")
        plt.bar(left_offset, batchY[batch_series_idx, :] * 0.5, width=1, color="red")
        plt.bar(left_offset, single_output_series * 0.3, width=1, color="green")

    plt.draw()
    plt.pause(0.0001)

def generateData():
    x = np.array(np.random.choice(2, total_series_length, p=[0.5, 0.5]))
    y = np.roll(x, echo_step)
    y[0:echo_step] = 0

    x = x.reshape((batch_size, -1))
    y = y.reshape((batch_size, -1))

    return (x, y)
    
class Test:
    def _init_graph(self):
        self.batchX_placeholder = tf.placeholder(tf.float32, [batch_size, truncated_backprop_length],name="xb")
        self.batchY_placeholder = tf.placeholder(tf.int32, [batch_size, truncated_backprop_length],name="yb")
        self.init_state = tf.placeholder(tf.float32, [batch_size, state_size],name="s0")
    
        self.W = tf.Variable(np.random.rand(state_size+1, state_size), dtype=tf.float32, name="W")
        self.b = tf.Variable(np.zeros((1,state_size)), dtype=tf.float32, name='b')
        self.W2 = tf.Variable(np.random.rand(state_size, num_classes),dtype=tf.float32, name="W2")
        self.b2 = tf.Variable(np.zeros((1,num_classes)), dtype=tf.float32, name="b2")
    
    def _forward(self):
        # Unpack columns
        self.inputs_series = tf.unstack(self.batchX_placeholder, axis=1)
        self.labels_series = tf.unstack(self.batchY_placeholder, axis=1)

        # Forward pass
        current_state = self.init_state
        states_series = []
        for current_input in self.inputs_series:
            current_input = tf.reshape(current_input, [batch_size, 1])
            input_and_state_concatenated = tf.concat([current_input, current_state], axis=1)

            next_state = tf.tanh(tf.matmul(input_and_state_concatenated, self.W) + self.b)  # Broadcasted addition
            states_series.append(next_state)
            current_state = next_state
        self.last_state = current_state
        self.logits_series = [tf.matmul(state, self.W2) + self.b2 for state in states_series]
        self.predictions_series = [tf.nn.softmax(logits) for logits in self.logits_series]
    
    def _train(self):
        losses = [tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels) for logits, labels in zip(self.logits_series, self.labels_series)]
        self.total_loss = tf.reduce_mean(losses)
        self.train_step = tf.train.AdagradOptimizer(0.3).minimize(self.total_loss)
    
    def train(self, x, y, n_epochs=100):
        self._init_graph()
        self._forward()
        self._train()
        
        nobs = len(x)
        divider = batch_size*truncated_backprop_length*n_epochs
        pad_len = divider - (nobs%divider)
        if nobs<pad_len:
            raise Exception("Wrong n_epochs({n_epochs}): choose smaller value.")
        
        x_batch=np.reshape(
            np.pad(x, (0, pad_len), mode='constant'),
            (n_epochs,batch_size, -1, truncated_backprop_length)
        )
        y_batch=np.reshape(
            np.pad(y, (0, pad_len), mode='constant'),
            (n_epochs,batch_size, -1, truncated_backprop_length)
        )
        n_batches=x_batch.shape[1]
        
        with tf.Session() as sess:
            sess.run(tf.initialize_all_variables())
            plt.ion()
            plt.figure()
            plt.show()
            loss_list = []

            for epoch_idx in range(num_epochs):
#                 x,y = generateData()
                _current_state = np.zeros((batch_size, state_size))

                print("New data, epoch", epoch_idx)

                for batch_idx in range(num_batches):
                    start_idx = batch_idx * truncated_backprop_length
                    end_idx = start_idx + truncated_backprop_length

#                     batchX = x[:,start_idx:end_idx]
#                     batchY = y[:,start_idx:end_idx]
                    
                    
                    _total_loss, _train_step, _current_state, _predictions_series = sess.run(
                        [self.total_loss, self.train_step, self.last_state, self.predictions_series],
                        feed_dict={
#                             self.batchX_placeholder:batchX,
#                             self.batchY_placeholder:batchY,
                            self.batchX_placeholder:x_batch[epoch_idx, :,batch_idx,:],
                            self.batchY_placeholder:y_batch[epoch_idx, :,batch_idx,:],
                            self.init_state:_current_state
                        }
                    )

                    loss_list.append(_total_loss)

                    if batch_idx%100 == 0:
                        print("Step",batch_idx, "Loss", _total_loss)
#                         plot(loss_list, _predictions_series, batchX, batchY)
#                         plot(loss_list, _predictions_series, x_batch[epoch_idx,:,batch_idx,:], y_batch[epoch_idx,:,batch_idx,:])
                        print(y_batch[epoch_idx,0, batch_idx, :])
                        print(np.array(_predictions_series)[:,0,:])

        plt.ioff()
        plt.show()

echo_step = 1
x = np.array(np.random.choice(2, 5000000, p=[0.5, 0.5]))
y = np.roll(x, echo_step)
y[0:echo_step] = 0
        
tst = Test()
tst.train(x,y)

<Figure size 432x288 with 0 Axes>

New data, epoch 0
Step 0 Loss 0.69896
[0 0 1 1 1 0 0 1 0 1 1 1 1 1 0]
[[ 0.5         0.5       ]
 [ 0.4686639   0.53133613]
 [ 0.44860706  0.55139297]
 [ 0.45085731  0.54914278]
 [ 0.45120201  0.54879802]
 [ 0.45117897  0.54882097]
 [ 0.45141613  0.54858381]
 [ 0.45128796  0.54871207]
 [ 0.45142812  0.54857188]
 [ 0.45149663  0.54850334]
 [ 0.45150131  0.54849869]
 [ 0.45150176  0.54849833]
 [ 0.45150176  0.54849821]
 [ 0.45129752  0.54870242]
 [ 0.45119309  0.54880685]]
Step 100 Loss 0.692492
[0 1 0 0 1 1 1 0 1 1 0 0 1 0 1]
[[ 0.47305369  0.52694631]
 [ 0.47323301  0.52676702]
 [ 0.47321156  0.52678841]
 [ 0.47305346  0.52694649]
 [ 0.47307798  0.52692211]
 [ 0.47307888  0.52692115]
 [ 0.4732348   0.52676523]
 [ 0.47305617  0.5269438 ]
 [ 0.4730781   0.52692193]
 [ 0.47323462  0.52676541]
 [ 0.47321197  0.52678806]
 [ 0.47305346  0.52694649]
 [ 0.47323301  0.52676702]
 [ 0.47305599  0.52694404]
 [ 0.47323331  0.52676666]]
Step 200 Loss 0.711866
[0 0 1 0 1 0 0 0 0 1 1 1 0 0 1]
[[ 0.432

Step 400 Loss 0.000916789
[1 1 1 1 0 0 1 1 1 0 1 1 1 1 0]
[[  8.56864732e-04   9.99143124e-01]
 [  8.44333379e-04   9.99155641e-01]
 [  8.47365474e-04   9.99152660e-01]
 [  9.88011714e-04   9.99011993e-01]
 [  9.99222398e-01   7.77583453e-04]
 [  9.99004304e-01   9.95665323e-04]
 [  8.55496561e-04   9.99144435e-01]
 [  8.44405033e-04   9.99155641e-01]
 [  9.88287386e-04   9.99011636e-01]
 [  9.98936236e-01   1.06380216e-03]
 [  8.30814592e-04   9.99169230e-01]
 [  8.46880372e-04   9.99153137e-01]
 [  8.47424380e-04   9.99152541e-01]
 [  9.88011714e-04   9.99011993e-01]
 [  9.98936236e-01   1.06379250e-03]]
Step 500 Loss 0.000840607
[1 1 0 0 0 1 1 0 1 1 0 0 0 0 1]
[[  7.99539499e-04   9.99200523e-01]
 [  9.13489785e-04   9.99086499e-01]
 [  9.99276102e-01   7.23884499e-04]
 [  9.99319673e-01   6.80313387e-04]
 [  9.99065220e-01   9.34743497e-04]
 [  8.01217044e-04   9.99198854e-01]
 [  9.13330470e-04   9.99086618e-01]
 [  9.99014020e-01   9.85963154e-04]
 [  7.75860390e-04   9.99224186e

Step 400 Loss 0.000443173
[1 0 1 0 1 0 0 0 0 1 0 0 0 1 0]
[[  4.74924222e-04   9.99525070e-01]
 [  9.99472082e-01   5.27936616e-04]
 [  4.60550596e-04   9.99539495e-01]
 [  9.99459088e-01   5.40882931e-04]
 [  4.55463713e-04   9.99544561e-01]
 [  9.99594629e-01   4.05355415e-04]
 [  9.99626637e-01   3.73319286e-04]
 [  9.99620914e-01   3.79114674e-04]
 [  9.99493361e-01   5.06665150e-04]
 [  4.82901261e-04   9.99517083e-01]
 [  9.99606550e-01   3.93431168e-04]
 [  9.99625444e-01   3.74535215e-04]
 [  9.99492884e-01   5.07145654e-04]
 [  4.82788746e-04   9.99517202e-01]
 [  9.99474466e-01   5.25578333e-04]]
Step 500 Loss 0.000422257
[1 1 0 0 0 1 1 0 0 1 1 1 1 0 1]
[[  4.16731404e-04   9.99583304e-01]
 [  4.57534392e-04   9.99542475e-01]
 [  9.99619961e-01   3.80037411e-04]
 [  9.99639392e-01   3.60562844e-04]
 [  9.99512553e-01   4.87427606e-04]
 [  4.16748488e-04   9.99583304e-01]
 [  4.57530667e-04   9.99542475e-01]
 [  9.99619961e-01   3.80037760e-04]
 [  9.99520898e-01   4.79073409e

Step 400 Loss 0.000298244
[0 1 0 1 0 1 0 0 0 1 1 1 1 1 0]
[[  9.99658704e-01   3.41251434e-04]
 [  3.03062407e-04   9.99696970e-01]
 [  9.99655247e-01   3.44723347e-04]
 [  2.94423022e-04   9.99705493e-01]
 [  9.99647856e-01   3.52136674e-04]
 [  2.91583448e-04   9.99708354e-01]
 [  9.99711335e-01   2.88658164e-04]
 [  9.99727190e-01   2.72792357e-04]
 [  9.99658585e-01   3.41454521e-04]
 [  2.87032744e-04   9.99713004e-01]
 [  2.81709421e-04   9.99718249e-01]
 [  2.82891764e-04   9.99717057e-01]
 [  2.82755034e-04   9.99717176e-01]
 [  2.98204861e-04   9.99701798e-01]
 [  9.99651551e-01   3.48426664e-04]]
Step 500 Loss 0.00029656
[0 1 0 0 1 1 1 0 0 0 0 0 1 0 0]
[[  9.99662519e-01   3.37457517e-04]
 [  2.87295814e-04   9.99712646e-01]
 [  9.99721229e-01   2.78808468e-04]
 [  9.99675274e-01   3.24783701e-04]
 [  2.81207089e-04   9.99718726e-01]
 [  2.74990569e-04   9.99725044e-01]
 [  2.91900855e-04   9.99708116e-01]
 [  9.99724090e-01   2.75881059e-04]
 [  9.99733508e-01   2.66430114e-

...

Step 400 Loss 2.71825e-05
[1 1 0 1 1 0 1 1 1 0 0 1 0 1 1]
[[  2.60134839e-05   9.99974012e-01]
 [  2.80611566e-05   9.99971986e-01]
 [  9.99971628e-01   2.84169528e-05]
 [  2.54490951e-05   9.99974608e-01]
 [  2.80161275e-05   9.99971986e-01]
 [  9.99971509e-01   2.84355465e-05]
 [  2.54362112e-05   9.99974608e-01]
 [  2.60904271e-05   9.99973893e-01]
 [  2.80297027e-05   9.99971986e-01]
 [  9.99973536e-01   2.64520695e-05]
 [  9.99970913e-01   2.91098568e-05]
 [  2.74752056e-05   9.99972582e-01]
 [  9.99971867e-01   2.81897392e-05]
 [  2.56038420e-05   9.99974370e-01]
 [  2.60087709e-05   9.99974012e-01]]
Step 500 Loss 2.72684e-05
[0 0 1 0 0 1 0 1 0 0 0 0 0 0 1]
[[  9.99972939e-01   2.70497167e-05]
 [  9.99970913e-01   2.90826647e-05]
 [  2.74011782e-05   9.99972582e-01]
 [  9.99973774e-01   2.61749356e-05]
 [  9.99970913e-01   2.91305187e-05]
 [  2.73736496e-05   9.99972582e-01]
 [  9.99971867e-01   2.81441826e-05]
 [  2.73633923e-05   9.99972582e-01]
 [  9.99973774e-01   2.62054054e

Step 400 Loss 2.63783e-05
[1 1 1 0 1 1 0 0 1 0 1 0 0 1 1]
[[  2.48604792e-05   9.99975085e-01]
 [  2.48619726e-05   9.99975085e-01]
 [  2.66980496e-05   9.99973297e-01]
 [  9.99971867e-01   2.81766224e-05]
 [  2.39312903e-05   9.99976039e-01]
 [  2.68560161e-05   9.99973178e-01]
 [  9.99973774e-01   2.62093308e-05]
 [  9.99971390e-01   2.85689785e-05]
 [  2.60120705e-05   9.99974012e-01]
 [  9.99971986e-01   2.80585882e-05]
 [  2.57220745e-05   9.99974251e-01]
 [  9.99973536e-01   2.64587052e-05]
 [  9.99971509e-01   2.84382586e-05]
 [  2.43015620e-05   9.99975681e-01]
 [  2.48312208e-05   9.99975204e-01]]
Step 500 Loss 2.61764e-05
[1 1 0 1 0 0 1 0 0 1 1 1 1 0 1]
[[  2.48173474e-05   9.99975204e-01]
 [  2.66361785e-05   9.99973416e-01]
 [  9.99971867e-01   2.80934710e-05]
 [  2.56056737e-05   9.99974370e-01]
 [  9.99973536e-01   2.64419305e-05]
 [  9.99971628e-01   2.83286336e-05]
 [  2.60354245e-05   9.99974012e-01]
 [  9.99973893e-01   2.60466477e-05]
 [  9.99971509e-01   2.85232181e

Step 400 Loss 2.55725e-05
[0 1 0 1 1 0 0 0 0 1 0 0 1 0 1]
[[  9.99974132e-01   2.58197651e-05]
 [  2.66857041e-05   9.99973297e-01]
 [  9.99975324e-01   2.46292457e-05]
 [  2.64260234e-05   9.99973536e-01]
 [  2.60561137e-05   9.99973893e-01]
 [  9.99975562e-01   2.44043731e-05]
 [  9.99975204e-01   2.47448424e-05]
 [  9.99975085e-01   2.49149998e-05]
 [  9.99974132e-01   2.58679738e-05]
 [  2.66588177e-05   9.99973297e-01]
 [  9.99976397e-01   2.36044252e-05]
 [  9.99973893e-01   2.61637069e-05]
 [  2.64959999e-05   9.99973536e-01]
 [  9.99975204e-01   2.47636781e-05]
 [  2.62891572e-05   9.99973655e-01]]
Step 500 Loss 2.55534e-05
[0 0 0 1 0 1 0 0 0 0 0 0 1 0 0]
[[  9.99975920e-01   2.40765821e-05]
 [  9.99975204e-01   2.48251126e-05]
 [  9.99974132e-01   2.58362688e-05]
 [  2.65593953e-05   9.99973416e-01]
 [  9.99975324e-01   2.46308900e-05]
 [  2.69666798e-05   9.99973059e-01]
 [  9.99976635e-01   2.33055925e-05]
 [  9.99974728e-01   2.52423106e-05]
 [  9.99975324e-01   2.46321833e

Step 400 Loss 2.48477e-05
[0 1 0 0 0 0 1 1 0 0 0 0 1 0 0]
[[  9.99974370e-01   2.56036237e-05]
 [  2.45738775e-05   9.99975443e-01]
 [  9.99975801e-01   2.42276074e-05]
 [  9.99975443e-01   2.44970124e-05]
 [  9.99975324e-01   2.47220360e-05]
 [  9.99973893e-01   2.60473171e-05]
 [  2.37815020e-05   9.99976277e-01]
 [  2.52242171e-05   9.99974728e-01]
 [  9.99975801e-01   2.42458900e-05]
 [  9.99975562e-01   2.44939092e-05]
 [  9.99975324e-01   2.47233784e-05]
 [  9.99973893e-01   2.60467968e-05]
 [  2.48895649e-05   9.99975085e-01]
 [  9.99976039e-01   2.39621877e-05]
 [  9.99973893e-01   2.60601391e-05]]
Step 500 Loss 2.46824e-05
[0 0 0 1 1 1 0 1 0 0 0 1 1 1 0]
[[  9.99975920e-01   2.41116468e-05]
 [  9.99975562e-01   2.44736657e-05]
 [  9.99973893e-01   2.60632696e-05]
 [  2.36951582e-05   9.99976277e-01]
 [  2.40290537e-05   9.99975920e-01]
 [  2.53257167e-05   9.99974728e-01]
 [  9.99974489e-01   2.55185751e-05]
 [  2.45419815e-05   9.99975443e-01]
 [  9.99975801e-01   2.41492926e

KeyboardInterrupt: 

In [31]:
def prepare_data(x,y,batch_sz,backprop_len,n_epochs):
    nobs = len(x)
    divider = batch_size*backprop_len*n_epochs
    pad_len = divider - (nobs%divider)
    if nobs<pad_len:
        raise Exception("Wrong n_epochs({n_epochs}): choose smaller value.")

    x_batch=np.reshape(
        np.pad(x, (0, pad_len), mode='constant'),
        (n_epochs,batch_sz, -1, backprop_len)
    )
    y_batch=np.reshape(
        np.pad(y, (0, pad_len), mode='constant'),
        (n_epochs,batch_sz, -1, backprop_len)
    )
    n_batches=x_batch.shape[1]
    return x_batch, y_batch, n_batches

In [32]:
xb,yb,n = prepare_data(x,y,batch_size,truncated_backprop_length,100)

In [36]:
x[:50]

array([1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1,
       1, 1, 0, 1])

In [34]:
xb[0,:,0,:]

array([[1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1],
       [1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0],
       [1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1],
       [1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0],
       [1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1]])

In [35]:
xb[0,:,1,:]

array([[0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1],
       [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1],
       [1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0]])

In [28]:
yb[0,1,:,:]

array([[1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1],
       [0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1],
       [0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1],
       [0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1],
       [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1]])

In [15]:
xb2,yb2 = generateData()

In [19]:
print(list(xb2[0]))

[0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 

In [20]:
print(list(yb2[0]))

[0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 

In [68]:
def generateData():
    x = np.array(np.random.choice(2, total_series_length, p=[0.5, 0.5]))
    y = np.roll(x, echo_step)
    y[0:echo_step] = 0

    x = x.reshape((batch_size, -1))
    y = y.reshape((batch_size, -1))

    return (x, y)

In [76]:
echo_step

1

In [69]:
x,y = generateData()

In [75]:
list(zip(x[0].tolist(),y[0].tolist()))

[(0, 0),
 (1, 0),
 (0, 1),
 (1, 0),
 (0, 1),
 (1, 0),
 (0, 1),
 (1, 0),
 (1, 1),
 (1, 1),
 (0, 1),
 (0, 0),
 (0, 0),
 (0, 0),
 (1, 0),
 (1, 1),
 (1, 1),
 (0, 1),
 (1, 0),
 (1, 1),
 (0, 1),
 (1, 0),
 (1, 1),
 (1, 1),
 (0, 1),
 (1, 0),
 (1, 1),
 (0, 1),
 (0, 0),
 (1, 0),
 (1, 1),
 (0, 1),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (1, 0),
 (1, 1),
 (0, 1),
 (1, 0),
 (0, 1),
 (1, 0),
 (0, 1),
 (0, 0),
 (1, 0),
 (1, 1),
 (1, 1),
 (1, 1),
 (0, 1),
 (1, 0),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (0, 1),
 (1, 0),
 (0, 1),
 (0, 0),
 (1, 0),
 (0, 1),
 (0, 0),
 (0, 0),
 (1, 0),
 (1, 1),
 (1, 1),
 (0, 1),
 (0, 0),
 (1, 0),
 (0, 1),
 (0, 0),
 (0, 0),
 (1, 0),
 (1, 1),
 (0, 1),
 (0, 0),
 (0, 0),
 (0, 0),
 (1, 0),
 (0, 1),
 (0, 0),
 (1, 0),
 (1, 1),
 (0, 1),
 (1, 0),
 (0, 1),
 (1, 0),
 (0, 1),
 (1, 0),
 (0, 1),
 (1, 0),
 (1, 1),
 (0, 1),
 (0, 0),
 (1, 0),
 (1, 1),
 (1, 1),
 (0, 1),
 (0, 0),
 (0, 0),
 (1, 0),
 (0, 1),
 (1, 0),
 (0, 1),
 (1, 0),
 (0, 1),
 (1, 0),
 (1, 1),
 (0, 1),
 (1, 0),
 (1, 1),
 

array([0, 0, 1, ..., 0, 1, 0])